This Code is written to implement two different machine learning algorithms for text classification.I have used a dataset of ML paper peer reviews from the International Conference of Learning Representation. Different performance metrices are visualised to compare the efficiency of the Algortihms for the given dataset.
Linear Support Vector Machine and Random Forest Classifier are trained with the given ML paper reviews dataset.

Importing all the necessary packages and libraries

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import pickle
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to C:\Users\Rabin Thapa
[nltk_data]     Kshetry\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Rabin Thapa
[nltk_data]     Kshetry\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Importing the Dataset

In [6]:
dataframe = pd.read_csv('C:/Users/Rabin Thapa Kshetry/Desktop/Term 2/AML/dataset/processed_reviews_split_surnamesABCD_minimal.csv', index_col=0)

Exploratory Data Analysis and Data Cleaning

In [7]:
# displaying the number of records in the dataset
print(len(dataframe))

6119


In [8]:
# displaying the statistical description of numerical attributes from the given dataset
dataframe.describe()

,confidence_score,review_score
count,3318.000000,6119.000000
mean,3.795057,4.689492
std,0.815260,2.192145
min,1.000000,-1.000000
25%,3.000000,3.000000
50%,4.000000,5.000000
75%,4.000000,6.000000
max,5.000000,10.000000


In [9]:
# displaying the datatypes of all the attributes from the given dataset
dataframe.dtypes

text                  object
confidence_score     float64
review_score         float64
acceptance_status     object
dtype: object

In [10]:
# identifying the duplicate records
print(dataframe.duplicated().sum())

271


In [11]:
# Dropping all the duplicates from the dataset and storing in the new dataframe dataframe1
dataframe1= dataframe.drop_duplicates()
print(len(dataframe1))

5848


In [12]:
# identifying the duplicate records for the text column specifically
print(dataframe1.text.duplicated().sum())

231


In [13]:
# Dropping all the duplicates from the dataset focussing in text column and storing in the new dataframe dataframe2
dataframe2= dataframe1.drop_duplicates(subset='text')
print(len(dataframe2))

5617


In [14]:
# Dropping all the records having missing values or labels and storing in the new dataframe dataframe3
dataframe3=dataframe2.dropna()
print(len(dataframe3))

3035


In [15]:
# Dropping all the records having invalid value for review_score column and storing in dataframe4
dataframe4 = dataframe3.drop(dataframe3.index[dataframe3['review_score'] == -1])
print(len(dataframe4))

2992


In [16]:
# Displaying the top 5 rows of the cleaned dataframe 
dataframe4.head(5)

,text,confidence_score,review_score,acceptance_status
review_id,,,,
iclr_review_0000,Predicting Medications from Diagnostic Codes w...,3.0,6.0,Accept
iclr_review_0003,Sufficient Conditions for Robustness to Advers...,3.0,5.0,Reject
iclr_review_0004,Unsupervised Learning of the Set of Local Maxi...,3.0,8.0,Accept
iclr_review_0005,Global Optimality Conditions for Deep Neural N...,4.0,7.0,Accept
iclr_review_0006,Gaussian Process Behaviour in Wide Deep Neural...,4.0,6.0,Accept


In [17]:
# Displaying the bottom 5 rows of the cleaned dataframe 
dataframe4.tail(5)

,text,confidence_score,review_score,acceptance_status
review_id,,,,
iclr_review_6108,Improving Composition of Sentence Embeddings t...,3.0,5.0,Reject
iclr_review_6109,Neural Tree Transducers for Tree to Tree Learn...,4.0,7.0,Reject
iclr_review_6112,Auxiliary Guided Autoregressive Variational Au...,4.0,5.0,Reject
iclr_review_6116,Learning to Augment Influential Data. Data aug...,4.0,6.0,Reject
iclr_review_6117,The Incredible Shrinking Neural Network: New P...,4.0,3.0,Reject


In [18]:
# Exporting the cleaned dataframe into a csv file in the local disk as exclusion4.csv
dataframe4.to_csv('C:/Users/Rabin Thapa Kshetry/Desktop/Term 2/AML/exclusion4.csv')

Target Label distribution into acceptance_status and review_score

In [87]:
# Allocating the attributes into feature variables(text) and target variables(acceptance_status) as x and y respectively
x= dataframe4.iloc[:,:-3].values
y= dataframe4.iloc[:,-1].values

#Displaying the values of x and y
print(x)
print(y)

[['Predicting Medications from Diagnostic Codes with Recurrent Neural Networks. It is a surprising fact that electronic medical records are failing at one of their primary purposes, that of tracking the set of medications that the patient is actively taking. Studies estimate that up to 50% of such lists omit active drugs, and that up to 25% of all active medications do not appear on the appropriate patient list. Manual efforts to maintain these lists involve a great deal of tedious human labor, which could be reduced by computational tools to suggest likely missing or incorrect medications on a patient’s list. We report here an application of recurrent neural networks to predict the likely therapeutic classes of medications that a patient is taking, given a sequence of the last 100 billing codes in their record. Our best model was a GRU that achieved high prediction accuracy (micro-averaged AUC 0.93, Label Ranking Loss 0.076), limited by hardware constraints on model size. Additionally

In [88]:
# Allocating the attributes into feature variables(text) and target variables(review_score) as x and y_rs respectively
x= dataframe4.iloc[:,:-3].values
y_rs= dataframe4.iloc[:,-2].values

#Displaying the values of x and y_rs
print(x)
print(y_rs)

[['Predicting Medications from Diagnostic Codes with Recurrent Neural Networks. It is a surprising fact that electronic medical records are failing at one of their primary purposes, that of tracking the set of medications that the patient is actively taking. Studies estimate that up to 50% of such lists omit active drugs, and that up to 25% of all active medications do not appear on the appropriate patient list. Manual efforts to maintain these lists involve a great deal of tedious human labor, which could be reduced by computational tools to suggest likely missing or incorrect medications on a patient’s list. We report here an application of recurrent neural networks to predict the likely therapeutic classes of medications that a patient is taking, given a sequence of the last 100 billing codes in their record. Our best model was a GRU that achieved high prediction accuracy (micro-averaged AUC 0.93, Label Ranking Loss 0.076), limited by hardware constraints on model size. Additionally

Text Pre-processing

Here the below code will remove all the punctuations, stopwords, unnecessary spaces and any special characters from the text.

In [89]:
texts = []

# Importing WordNetLemmatizer from nltk.stem
from nltk.stem import WordNetLemmatizer

# Creating object of WordNetLemmatizer
stemmer = WordNetLemmatizer()

# Executing for loop to iterate through all the text and remove the unnecessary characters
for sen in range(0, len(x)):
    # this removes all the special characters
    text = re.sub(r'\W', ' ', str(x[sen]))
    
    # this removes all single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    
    # this removes single characters from the start
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text) 
    
    # this substitutes all multiple spaces with single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    
    # this removes prefixed 'b'
    text = re.sub(r'^b\s+', '', text)
    
    # this converts all the texts to Lowercase
    text = text.lower()
    
    # Lemmatization
    text = text.split()

    text = [stemmer.lemmatize(word) for word in text]
    text = ' '.join(text)
    
    texts.append(text)

In [90]:
# Importing CountVectorizer class from sklearn.feature_extraction.text library
from sklearn.feature_extraction.text import CountVectorizer

# this code converts all the texts into corresponding numerical features
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
x = vectorizer.fit_transform(texts).toarray()

In [91]:
# Importing TfidTransformer class from sklearn.feature_extraction.text library
from sklearn.feature_extraction.text import TfidfTransformer

# this code converts all the text into TFID feature values
# TFID stands for term frequency and Inverse Document frequency
tfidfconverter = TfidfTransformer()
x = tfidfconverter.fit_transform(x).toarray()

Data Splitting

In [92]:
# Importing train_test_split utility from sklearn.model_selection 
from sklearn.model_selection import train_test_split

# Splitting the dataset into training and test set
# here training set contains 80% of the data whereas test set contains just 20% of the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)
X_train_rs, X_test_rs, y_train_rs, y_test_rs = train_test_split(x,y_rs, test_size=0.2, random_state=0) # for review_score

Training the Models ( Linear Support Vector Machine and Random Forest Classifier)

In [93]:
# Importing the Support Vector Machine from sklearn
from sklearn import svm
clf = svm.SVC(kernel='linear') # for acceptance_status
clf_rs = svm.SVC(kernel='linear') # for review_score

# Training the model with training data
clf.fit(X_train, y_train)
clf_rs.fit(X_train_rs, y_train_rs)

SVC(kernel='linear')

In [94]:
# Importing RandomForestClassifier from sklearn.ensemble library
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=1000, random_state=0) # for acceptance_status
classifier_rs = RandomForestClassifier(n_estimators=1000, random_state=0) # for review_score

# Training the Algorithm with training dataset
classifier.fit(X_train, y_train) # for acceptance_status
classifier_rs.fit(X_train_rs, y_train_rs) # for review_score

RandomForestClassifier(n_estimators=1000, random_state=0)

In [95]:
# Predicting the target value for test set using SVM Model
y_pred1 = clf.predict(X_test) # for acceptance_status
y_pred1_rs = clf_rs.predict(X_test_rs) # for review_score

# predicting the target value for the test set using Random Forest Classifier Model
y_pred2 = classifier.predict(X_test) # for acceptance_status
y_pred2_rs = classifier_rs.predict(X_test_rs) # for review_score


Evaluating Both the Models

In [96]:
# Importing utilities Classification_report, confusion_matrix and accuracy_score from sklearn.metrics library
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Displaying the results of Evaluation metrices for SVM Model for acceptance_status
print('Evaluation metrices for SVM Model for attribute acceptance_status')
print(confusion_matrix(y_test,y_pred1))
print(classification_report(y_test,y_pred1, zero_division=0))
print(accuracy_score(y_test, y_pred1))
print('-------------------------------------------------------')

# Displaying the results of Evaluation metrices for SVM Model for review_score
print('Evaluation metrices for SVM Model for attribute review_score')
print(confusion_matrix(y_test_rs,y_pred1_rs))
print(classification_report(y_test_rs,y_pred1_rs, zero_division=0))
print(accuracy_score(y_test_rs, y_pred1_rs))
print('-------------------------------------------------------')

# Displaying the results of Evaluation metrices for Random Forest Classifier Model for acceptance_status
print('Evaluation metrices for Random Forest Classifier Model for attribute acceptance_status')
print(confusion_matrix(y_test,y_pred2))
print(classification_report(y_test,y_pred2, zero_division=0))
print(accuracy_score(y_test, y_pred2))
print('---------------------------------------------------------')

# Displaying the results of Evaluation metrices for Random Forest Classifier Model for review_score
print('Evaluation metrices for Random Forest Classifier Model for attribute review_score')
print(confusion_matrix(y_test_rs,y_pred2_rs))
print(classification_report(y_test_rs,y_pred2_rs, zero_division=0))
print(accuracy_score(y_test_rs, y_pred2_rs))

Evaluation metrices for SVM Model for attribute acceptance_status
[[109 128]
 [ 61 301]]
              precision    recall  f1-score   support

      Accept       0.64      0.46      0.54       237
      Reject       0.70      0.83      0.76       362

    accuracy                           0.68       599
   macro avg       0.67      0.65      0.65       599
weighted avg       0.68      0.68      0.67       599

0.6844741235392321
-------------------------------------------------------
Evaluation metrices for SVM Model for attribute review_score
[[ 0  1  3  6  2  2  0  0]
 [ 0  2 15 15  8  8  0  0]
 [ 0  2 26 41 37 11  0  0]
 [ 0  1 27 28 53 12  0  0]
 [ 0  1 31 31 57 19  0  0]
 [ 0  2 15 25 44 31  0  0]
 [ 0  0  6  4 14 12  0  0]
 [ 0  0  2  0  2  3  0  0]]
              precision    recall  f1-score   support

         2.0       0.00      0.00      0.00        14
         3.0       0.22      0.04      0.07        48
         4.0       0.21      0.22      0.21       117
         5.0  

Validation of the Models for predicting both the attributes

In [97]:
# Creating a Validation set and validating the model
train_ratio = 0.80
test_ratio = 0.10
validation_ratio = 0.10

# Creating training and test set for both attributes
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=test_ratio) # for acceptance_status
X_train_rs, X_test_rs, y_train_rs, y_test_rs = train_test_split(x, y_rs, test_size=test_ratio) # for review_score

# Creating training and validation set for both attributes
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=validation_ratio/(train_ratio+test_ratio)) # for acceptance_status
X_train_rs, X_valid_rs, y_train_rs, y_valid_rs = train_test_split(X_train_rs, y_train_rs, test_size=validation_ratio/(train_ratio+test_ratio)) # for review_score

# displaying the shape of training, test and Validation set for attribute acceptance_status
print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)
print('----------------------------------------------------------------------------')

# displaying the shape of training, test and Validation set for attribute review_score
print(X_train_rs.shape)
print(X_test_rs.shape)
print(X_valid_rs.shape)

(2392, 1500)
(300, 1500)
(300, 1500)
----------------------------------------------------------------------------
(2392, 1500)
(300, 1500)
(300, 1500)


In [98]:
# predicting the target values for the Validation set 

# for SVM Model
y_pred3 = clf.predict(X_valid) # for acceptance_status
y_pred3_rs = clf_rs.predict(X_valid_rs) # for review_score

# for Random Forest Classifier
y_pred4 = classifier.predict(X_valid) # for acceptance_status
y_pred4_rs = classifier_rs.predict(X_valid_rs) # for review_score

In [99]:
# Displaying the results of Evaluation metrices for validation set using SVM Model for acceptance_status
print('Evaluation metrices for validation set of attribute acceptance_status using SVM Model')
print(confusion_matrix(y_valid,y_pred3))
print(classification_report(y_valid,y_pred3, zero_division=0))
print(accuracy_score(y_valid, y_pred3))
print('------------------------------------------------------------')

# Displaying the results of Evaluation metrices for validation set using SVM Model for review_score
print('Evaluation metrices for validation set of attribute review_score using SVM Model')
print(confusion_matrix(y_valid_rs,y_pred3_rs))
print(classification_report(y_valid_rs,y_pred3_rs, zero_division=0))
print(accuracy_score(y_valid_rs, y_pred3_rs))
print('------------------------------------------------------------')

# Displaying the results of Evaluation metrices for validation set using Random Forest Classifier for acceptance_status
print('Evaluation metrices for validation set of attribute acceptance_status using Random Forest Classifier')
print(confusion_matrix(y_valid,y_pred4))
print(classification_report(y_valid,y_pred4, zero_division=0))
print(accuracy_score(y_valid, y_pred4))
print('--------------------------------------------------------------')

# Displaying the results of Evaluation metrices for validation set using Random Forest Classifier for review_score
print('Evaluation metrices for validation set of attribute review_score using Random Forest Classifier')
print(confusion_matrix(y_valid_rs,y_pred4_rs))
print(classification_report(y_valid_rs,y_pred4_rs, zero_division=0))
print(accuracy_score(y_valid_rs, y_pred4_rs))
print('--------------------------------------------------------------')

Evaluation metrices for validation set of attribute acceptance_status using SVM Model
[[ 82  39]
 [ 20 159]]
              precision    recall  f1-score   support

      Accept       0.80      0.68      0.74       121
      Reject       0.80      0.89      0.84       179

    accuracy                           0.80       300
   macro avg       0.80      0.78      0.79       300
weighted avg       0.80      0.80      0.80       300

0.8033333333333333
------------------------------------------------------------
Evaluation metrices for validation set of attribute review_score using SVM Model
[[ 0  0  0  0  1  0  1  0  0]
 [ 0  0  0  1  1  0  0  0  0]
 [ 0  0  8  9  3  5  1  0  0]
 [ 0  0  0 41 12 13  3  0  0]
 [ 0  0  0  9 29 12  5  0  0]
 [ 0  0  0  8  6 50  5  0  0]
 [ 0  0  1  6  9 10 28  0  0]
 [ 0  0  0  3  4  6  3  1  0]
 [ 0  0  0  0  0  2  3  0  1]]
              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         2
         2.0       0.00